In [1]:
import os
import numpy as np

from fs2r import BASE_DIR
from fs2r.robot_env import RobotEnv
from fs2r.utils.pointcloud import read_calibration_file

calib_file = os.path.join(BASE_DIR, "perception/calibrations", "most_recent_calib.json")
calib_dict = read_calibration_file(calib_file)
env_config = {
    "address": "172.16.0.1",
    "port": 5050,
    "camera_index": 0,
    "obs_keys": ["lang_instr", "qpos", "qpos_normalized", "gripper_state_discrete", "rgb", "depth", "camera_intrinsic", "camera_extrinsic"],
    "calib_dict": calib_dict,
    "foundation_stereo": True,
    "img_resize": (128, 128),
}
env = RobotEnv(**env_config)
obs = env.reset()

def depth_to_points(depth, intrinsic, extrinsic, depth_scale=1000.0):
    height, width = depth.shape[:2]
    depth = depth.squeeze() / depth_scale
    xlin = np.linspace(0, width - 1, width)
    ylin = np.linspace(0, height - 1, height)
    px, py = np.meshgrid(xlin, ylin)
    px = (px - intrinsic[0, 2]) * (depth / intrinsic[0, 0])
    py = (py - intrinsic[1, 2]) * (depth / intrinsic[1, 1])
    points = np.stack((px, py, depth, np.ones(depth.shape)), axis=-1)
    points = (extrinsic @ points.reshape(-1, 4).T).T
    points = points[:, :3]
    return points

from fs2r.utils.meshcat import create_visualizer, visualize_pointcloud
vis = create_visualizer()

obs_real = env.get_obs()
H, W = obs_real["depth"].shape
points = depth_to_points(obs_real["depth"].reshape(H, W), obs_real["camera_intrinsic"].reshape(3, 3), obs_real["camera_extrinsic"].reshape(4, 4), depth_scale=1000.)
points = points.reshape(H, W, 3)
colors = obs_real["rgb"].reshape(H, W, 3) / 255.

visualize_pointcloud(
    vis, 'points_real',
    pc=points,
    color=colors * 255.,
    # color=np.ones_like(points) * [0., 0., 255.],
    size=0.01
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Waiting for meshcat server... have you started a server? Run `meshcat-server` to start a server
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
